# Test of a Swap Contract

We introduce new variants of `marlowe-cli run execute` and `marlowe-cli run withdraw` that automatically handle the UTxOs management, coin selection, and balancing, so that one doesn't need to specify `--tx-in` and `--tx-out` for Marlowe transactions. *These commands are experimental, and lightly tested so far.*

In [1]:
marlowe-cli run auto-execute --help

Usage: marlowe-cli run auto-execute 
         --testnet-magic INTEGER --socket-path SOCKET_FILE 
         [--marlowe-in-file MARLOWE_FILE --tx-in-marlowe TXID#TXIX]
         --marlowe-out-file MARLOWE_FILE --change-address ADDRESS 
         [--required-signer SIGNING_FILE] [--metadata-file METADATA_FILE]
         --out-file FILE [--submit SECONDS] [--print-stats] [--script-invalid]

  [EXPERIMENTAL] Run a Marlowe transaction, selecting transaction inputs and
  outputs automatically.

Available options:
  --testnet-magic INTEGER  Network magic. Defaults to the CARDANO_TESTNET_MAGIC
                           environment variable's value.
  --socket-path SOCKET_FILE
                           Location of the cardano-node socket file. Defaults to
                           the CARDANO_NODE_SOCKET_PATH environment variable's
                           value.
  --marlowe-in-file MARLOWE_FILE
                           JSON file with the Marlowe initial state and initial
                    

In [2]:
marlowe-cli run auto-withdraw --help

Usage: marlowe-cli run auto-withdraw 
         --testnet-magic INTEGER --socket-path SOCKET_FILE
         --marlowe-file MARLOWE_FILE --role-name TOKEN_NAME
         --change-address ADDRESS [--required-signer SIGNING_FILE] 
         [--metadata-file METADATA_FILE] --out-file FILE [--submit SECONDS] 
         [--print-stats] [--script-invalid]

  [EXPERIMENTAL] Withdraw funds from the Marlowe role address, selecting
  transaction inputs and outputs automatically.

Available options:
  --testnet-magic INTEGER  Network magic. Defaults to the CARDANO_TESTNET_MAGIC
                           environment variable's value.
  --socket-path SOCKET_FILE
                           Location of the cardano-node socket file. Defaults to
                           the CARDANO_NODE_SOCKET_PATH environment variable's
                           value.
  --marlowe-file MARLOWE_FILE
                           JSON file with the Marlowe inputs, final state, and
                           final contract.
 

## Preliminaries

Record version numbers

In [3]:
marlowe-cli --version

marlowe-cli 0.0.7.0 @ 25f6808de7aca1ab3d47db52fb4b66cae4228618


In [4]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


In [5]:
git rev-parse HEAD

25f6808de7aca1ab3d47db52fb4b66cae4228618


Set the location of keys.

In [6]:
TREASURY=treasury

Set the location of the node socket.

In [7]:
export CARDANO_NODE_SOCKET_PATH=node.socket

## Preliminaries

### Setup the faucet.

In [8]:
FAUCET_SKEY_FILE=$TREASURY/payment.skey
FAUCET_ADDRESS=$(cat $TREASURY/payment.testnet.address)
echo "$FAUCET_ADDRESS"

addr_test1vq9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupczgtm9j


### Select Network

In [9]:
export CARDANO_TESTNET_MAGIC=2

Compute the slot-to-time parameters.

In [10]:
SLOT_LENGTH=$(marlowe-cli util slotting --testnet-magic "$CARDANO_TESTNET_MAGIC" --socket-path "$CARDANO_NODE_SOCKET_PATH" | jq .scSlotLength)
SLOT_OFFSET=$(marlowe-cli util slotting --testnet-magic "$CARDANO_TESTNET_MAGIC" --socket-path "$CARDANO_NODE_SOCKET_PATH" | jq .scSlotZeroTime)

### Tip of the Blockchain

In [11]:
TIP=$(cardano-cli query tip --testnet-magic "$CARDANO_TESTNET_MAGIC" | jq '.slot')
NOW="$((TIP*SLOT_LENGTH+SLOT_OFFSET))"
HOUR="$((3600*1000))"

The tip of the blockchain:

In [12]:
echo $TIP

2304024


The current POSIX time implies that the tip of the blockchain should be slightly before the following slot:

In [13]:
echo $(((1000 * $(date -u +%s) - SLOT_OFFSET) / SLOT_LENGTH))

2304030


Tests main fail if this is not the case.

### Participants

#### The First Party

In [14]:
PARTY_A_PREFIX="$TREASURY/john-fletcher"
PARTY_A_PAYMENT_SKEY="$PARTY_A_PREFIX".skey
PARTY_A_PAYMENT_VKEY="$PARTY_A_PREFIX".vkey

Create the first party's keys, if necessary.

In [15]:
if [[ ! -e "$PARTY_A_PAYMENT_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_A_PAYMENT_SKEY"      \
                              --verification-key-file "$PARTY_A_PAYMENT_VKEY"
fi
PARTY_A_ADDRESS=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$PARTY_A_PAYMENT_VKEY" )
PARTY_A_PUBKEYHASH=$(cardano-cli address key-hash --payment-verification-key-file "$PARTY_A_PAYMENT_VKEY")

Address of the first party and its public key hash:

In [16]:
echo "Address: $PARTY_A_ADDRESS"
echo "Public key hash: $PARTY_A_PUBKEYHASH"

Address: addr_test1vqwt2xlr4d8yk4qws675exlqy6pdhq2s76wrehkjggkvr0cerfe8r
Public key hash: 1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bf


Fund the first party's address.

In [17]:
marlowe-cli util faucet --out-file /dev/null                      \
                        --submit 600                              \
                        --lovelace 100000000                      \
                        --faucet-address "$FAUCET_ADDRESS"        \
                        --required-signer "$FAUCET_SKEY_FILE"     \
                        "$PARTY_A_ADDRESS"

TxId "3336061b4e073083fe2c3c4562b478d34de0028ff2f67f143a0191056000d4f2"


The first party mints their tokens for the swap.

In [18]:
MINT_EXPIRES=$((TIP + 1000000))

In [19]:
TOKEN_NAME_A=Globe
AMOUNT_A=300
CURRENCY_SYMBOL_A=$(
marlowe-cli util mint --required-signer "$PARTY_A_PAYMENT_SKEY" \
                      --change-address  "$PARTY_A_ADDRESS"      \
                      --count "$AMOUNT_A"                       \
                      --expires "$MINT_EXPIRES"                 \
                      --out-file /dev/null                      \
                      --submit=600                              \
                      "$TOKEN_NAME_A"                           \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                            \
)
TOKEN_A="$CURRENCY_SYMBOL_A.$TOKEN_NAME_A"

They have the following UTxOs in their wallet:

In [20]:
marlowe-cli util clean --required-signer "$PARTY_A_PAYMENT_SKEY" \
                       --change-address "$PARTY_A_ADDRESS"       \
                       --out-file /dev/null                      \
                       --submit=600                              \
> /dev/null
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PARTY_A_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
7684d3181842c19b83c974ae7a13b5cb9550c9b2e5be5a6661c0c40e823e2cad     0        97651266 lovelace + TxOutDatumNone
7684d3181842c19b83c974ae7a13b5cb9550c9b2e5be5a6661c0c40e823e2cad     1        2000000 lovelace + 300 4b6897a0af19de09282bad1b3d191ace3c61ec364aa009102f0d23b3.476c6f6265 + TxOutDatumNone


The first party will trade the following in the swap:

In [21]:
echo "Amount: $AMOUNT_A"
echo "Token: $TOKEN_A"

Amount: 300
Token: 4b6897a0af19de09282bad1b3d191ace3c61ec364aa009102f0d23b3.Globe


#### The Second Party

In [22]:
PARTY_B_PREFIX="$TREASURY/thomas-kyd"
PARTY_B_PAYMENT_SKEY="$PARTY_B_PREFIX".skey
PARTY_B_PAYMENT_VKEY="$PARTY_B_PREFIX".vkey

Create the second party's keys, if necessary.

In [23]:
if [[ ! -e "$PARTY_B_PAYMENT_SKEY" ]]
then
  cardano-cli address key-gen --signing-key-file "$PARTY_B_PAYMENT_SKEY"      \
                              --verification-key-file "$PARTY_B_PAYMENT_VKEY"
fi
PARTY_B_ADDRESS=$(cardano-cli address build --testnet-magic "$CARDANO_TESTNET_MAGIC" --payment-verification-key-file "$PARTY_B_PAYMENT_VKEY" )
PARTY_B_PUBKEYHASH=$(cardano-cli address key-hash --payment-verification-key-file "$PARTY_B_PAYMENT_VKEY")

Address of the second party and its public key hash:

In [24]:
echo "Address: $PARTY_B_ADDRESS"
echo "Public key hash: $PARTY_B_PUBKEYHASH"

Address: addr_test1vr7n0zzth5zycuh972w7rdmh48qur4f3wu6ntn2m2h30dlcvltuy5
Public key hash: fd37884bbd044c72e5f29de1b777a9c1c1d531773535cd5b55e2f6ff


Fund the second party's address.

In [25]:
marlowe-cli util faucet --out-file /dev/null                  \
                        --submit 600                          \
                        --lovelace 100000000                  \
                        --faucet-address "$FAUCET_ADDRESS"    \
                        --required-signer "$FAUCET_SKEY_FILE" \
                        "$PARTY_B_ADDRESS"

TxId "0b20a7dc50288b3b824f6d3d6cde7c643c535add4c9f272b03d791bda5314c07"


The second party mints their tokens for the swap.

In [26]:
TOKEN_NAME_B=Swan
AMOUNT_B=500
CURRENCY_SYMBOL_B=$(
marlowe-cli util mint --required-signer "$PARTY_B_PAYMENT_SKEY" \
                      --change-address  "$PARTY_B_ADDRESS"      \
                      --count "$AMOUNT_B"                       \
                      --expires "$MINT_EXPIRES"                 \
                      --out-file /dev/null                      \
                      --submit=600                              \
                      "$TOKEN_NAME_B"                           \
| sed -e 's/^PolicyID "\(.*\)"$/\1/'                            \
)
TOKEN_B="$CURRENCY_SYMBOL_B.$TOKEN_NAME_B"

They have the following UTxOs in their wallet:

In [27]:
marlowe-cli util clean --required-signer "$PARTY_B_PAYMENT_SKEY" \
                       --change-address "$PARTY_B_ADDRESS"       \
                       --out-file /dev/null                      \
                       --submit=600                              \
> /dev/null
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PARTY_B_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ff8f2b3edd5ebd121f5a4792a9da177818cc1f319a253fbbea68d9a24d50ccb2     0        97651398 lovelace + TxOutDatumNone
ff8f2b3edd5ebd121f5a4792a9da177818cc1f319a253fbbea68d9a24d50ccb2     1        2000000 lovelace + 500 13d32b28fbb50440a92f618434d91a6bdbf46d4da9a7838a82937492.5377616e + TxOutDatumNone


The second party will trade the following in the swap:

In [28]:
echo "Amount: $AMOUNT_B"
echo "Token: $TOKEN_B"

Amount: 500
Token: 13d32b28fbb50440a92f618434d91a6bdbf46d4da9a7838a82937492.Swan


## The Contract

The initial Ada in the contract.

In [29]:
MINIMUM_ADA=3000000

In [30]:
PARTY_A_TIMEOUT=$((NOW+12*HOUR))
PARTY_B_TIMEOUT=$((NOW+24*HOUR))

The timeout for the first party:

In [31]:
echo $(date -u -R -d @$((PARTY_A_TIMEOUT/1000)))

Mon, 05 Sep 2022 04:00:24 +0000


The timeout for the second party:

In [32]:
echo $(date -u -R -d @$((PARTY_B_TIMEOUT/1000)))

Mon, 05 Sep 2022 16:00:24 +0000


We create the contract for the previously specified parameters.

In [33]:
marlowe-cli template swap --minimum-ada "$MINIMUM_ADA"       \
                          --a-party "PK=$PARTY_A_PUBKEYHASH" \
                          --a-token "$TOKEN_A"               \
                          --a-amount "$AMOUNT_A"             \
                          --a-timeout "$PARTY_A_TIMEOUT"     \
                          --b-party "PK=$PARTY_B_PUBKEYHASH" \
                          --b-token "$TOKEN_B"               \
                          --b-amount "$AMOUNT_B"             \
                          --b-timeout "$PARTY_B_TIMEOUT"     \
                          --out-contract-file tx-1.contract  \
                          --out-state-file    tx-1.state

## Transaction 1. Create the Contract by Providing the Minimum ADA.

First we create a `.marlowe` file that contains the initial information needed to run the contract. The bare size and cost of the script provide a lower bound on the resources that running it will require.

In [34]:
marlowe-cli run initialize --contract-file tx-1.contract \
                           --state-file    tx-1.state    \
                           --out-file      tx-1.marlowe  \
                           --print-stats


Validator size: 12383
Base-validator cost: ExBudget {exBudgetCPU = ExCPU 18768100, exBudgetMemory = ExMemory 81700}


In particular, we can extract the contract's address from the `.marlowe` file.

In [35]:
CONTRACT_ADDRESS=$(jq -r '.tx.marloweValidator.address' tx-1.marlowe)

The Marlowe contract resides at the following address:

In [36]:
echo $CONTRACT_ADDRESS

addr_test1wz0znl0lfq7jqmgduzgsu7ndnjt4npgeg5a75j3upvn7dsq3dwfjv


The first party submits the transaction along with the minimum Ada required for the contract's initial state. Submitting with the `--print-stats` switch reveals the network fee for the contract, the size of the transaction, and the execution requirements, relative to the protocol limits.

In [37]:
TX_1=$(
marlowe-cli run auto-execute --required-signer "$PARTY_A_PAYMENT_SKEY" \
                             --marlowe-out-file tx-1.marlowe           \
                             --change-address "$PARTY_A_ADDRESS"       \
                             --out-file tx-1.raw                       \
                             --print-stats                             \
                             --submit 600                              \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                       \
)


Fee: Lovelace 208621
Size: 864 / 16384 = 5%
Execution units:
  Memory: 0 / 14000000 = 0%
  Steps: 0 / 10000000000 = 0%


The contract received the minimum Ada from the first party in the following transaction:

In [38]:
echo $TX_1

641b465c7e088580f7043a99a9d4b15651ded6f62a9c3183400e747f6f9d4810


The following UTxOs are at the contract address:

In [39]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$CONTRACT_ADDRESS" | sed -n -e "1p;2p;/$TX_1/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
641b465c7e088580f7043a99a9d4b15651ded6f62a9c3183400e747f6f9d4810     1        3000000 lovelace + TxOutDatumHash ScriptDataInBabbageEra "ffe678ec0072c21067c86a074609d0919f1aaa297dc6a7c4a35f40ecc8638645"


Here are the UTxOs at the first party address:

In [40]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PARTY_A_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
641b465c7e088580f7043a99a9d4b15651ded6f62a9c3183400e747f6f9d4810     0        94442645 lovelace + TxOutDatumNone
7684d3181842c19b83c974ae7a13b5cb9550c9b2e5be5a6661c0c40e823e2cad     1        2000000 lovelace + 300 4b6897a0af19de09282bad1b3d191ace3c61ec364aa009102f0d23b3.476c6f6265 + TxOutDatumNone


## Transaction 2. First Party Deposits Tokens into the Contract.

First we compute the Marlowe input required to deposit the tokens.

In [41]:
marlowe-cli run prepare --marlowe-file tx-1.marlowe                \
                        --deposit-account "PK=$PARTY_A_PUBKEYHASH" \
                        --deposit-party "PK=$PARTY_A_PUBKEYHASH"   \
                        --deposit-token "$TOKEN_A"                 \
                        --deposit-amount "$AMOUNT_A"               \
                        --invalid-before "$NOW"                    \
                        --invalid-hereafter "$((NOW+4*HOUR))"      \
                        --out-file tx-2.marlowe                    \
                        --print-stats


Datum size: 540


Now the first party submits the transaction that deposits their tokens.

In [42]:
TX_2=$(
marlowe-cli run auto-execute --marlowe-in-file tx-1.marlowe            \
                             --tx-in-marlowe "$TX_1#1"                 \
                             --required-signer "$PARTY_A_PAYMENT_SKEY" \
                             --marlowe-out-file tx-2.marlowe           \
                             --change-address "$PARTY_A_ADDRESS"       \
                             --out-file tx-2.raw                       \
                             --print-stats                             \
                             --submit=600                              \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                       \
)


Fee: Lovelace 1142704
Size: 14051 / 16384 = 85%
Execution units:
  Memory: 4535828 / 14000000 = 32%
  Steps: 1215052593 / 10000000000 = 12%


The contract received the deposit of the first party's token for the swap in the following transaction:

In [43]:
echo $TX_2

ca39e65cbf587540c455bc319c3154897408f57f34ea0547f8e40fc84bba028c


The following UTxOs are at the contract address:

In [44]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$CONTRACT_ADDRESS" | sed -n -e "1p;2p;/$TX_1/p;/$TX_2/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ca39e65cbf587540c455bc319c3154897408f57f34ea0547f8e40fc84bba028c     1        3000000 lovelace + 300 4b6897a0af19de09282bad1b3d191ace3c61ec364aa009102f0d23b3.476c6f6265 + TxOutDatumHash ScriptDataInBabbageEra "e66a694645d6e6864e6484ae8c4035fc14bef19401bbee67e12ce07a6e5b1555"


Here are the UTxOs at the first party address:

In [45]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PARTY_A_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
ca39e65cbf587540c455bc319c3154897408f57f34ea0547f8e40fc84bba028c     0        95299941 lovelace + TxOutDatumNone


## Transaction 3. The Second Party Deposits their Tokens to Complete the Swap.

In [46]:
marlowe-cli run prepare --marlowe-file tx-2.marlowe                \
                        --deposit-account "PK=$PARTY_B_PUBKEYHASH" \
                        --deposit-party "PK=$PARTY_B_PUBKEYHASH"   \
                        --deposit-token "$TOKEN_B"                 \
                        --deposit-amount "$AMOUNT_B"               \
                        --invalid-before "$NOW"                    \
                        --invalid-hereafter "$((NOW+4*HOUR))"      \
                        --out-file tx-3.marlowe                    \
                        --print-stats


Datum size: 25
Payment 1
  Acccount: PK "1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bf"
  Payee: Party (PK "fd37884bbd044c72e5f29de1b777a9c1c1d531773535cd5b55e2f6ff")
  Ada: 0.000000
  4b6897a0af19de09282bad1b3d191ace3c61ec364aa009102f0d23b3."Globe": 300
Payment 2
  Acccount: PK "fd37884bbd044c72e5f29de1b777a9c1c1d531773535cd5b55e2f6ff"
  Payee: Party (PK "1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bf")
  Ada: 0.000000
  13d32b28fbb50440a92f618434d91a6bdbf46d4da9a7838a82937492."Swan": 500
Payment 3
  Acccount: PK "1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bf"
  Payee: Party (PK "1cb51be3ab4e4b540e86bd4c9be02682db8150f69c3cded2422cc1bf")
  Ada: 3.000000


Now the second party can submit a transaction that deposits their tokens and completes the swap.

In [47]:
TX_3=$(
marlowe-cli run auto-execute --marlowe-in-file tx-2.marlowe            \
                             --tx-in-marlowe "$TX_2#1"                 \
                             --required-signer "$PARTY_B_PAYMENT_SKEY" \
                             --marlowe-out-file tx-3.marlowe           \
                             --change-address "$PARTY_B_ADDRESS"       \
                             --out-file tx-3.raw                       \
                             --print-stats                             \
                             --submit=600                              \
| sed -e 's/^TxId "\(.*\)"$/\1/'                                       \
)


Fee: Lovelace 1225823
Size: 13503 / 16384 = 82%
Execution units:
  Memory: 5958314 / 14000000 = 42%
  Steps: 1563921919 / 10000000000 = 15%


The closing of the contract paid the swapped tokens to the parties in the transaction:

In [48]:
echo $TX_3

051fb631e56a88f80c098a56ccd5227513d77ca03d9be7d7e166c7ce1f810f57


There is no UTxO at the contract address:

In [49]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$CONTRACT_ADDRESS" | sed -n -e "1p;2p;/$TX_1/p;/$TX_2/p;/$TX_3/p"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


Here are the UTxOs at the first party's address:

In [50]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PARTY_A_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
051fb631e56a88f80c098a56ccd5227513d77ca03d9be7d7e166c7ce1f810f57     1        3000000 lovelace + 500 13d32b28fbb50440a92f618434d91a6bdbf46d4da9a7838a82937492.5377616e + TxOutDatumNone
ca39e65cbf587540c455bc319c3154897408f57f34ea0547f8e40fc84bba028c     0        95299941 lovelace + TxOutDatumNone


Here are the UTxOs at the second party's address:

In [51]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PARTY_B_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
051fb631e56a88f80c098a56ccd5227513d77ca03d9be7d7e166c7ce1f810f57     0        97386865 lovelace + TxOutDatumNone
051fb631e56a88f80c098a56ccd5227513d77ca03d9be7d7e166c7ce1f810f57     2        1038710 lovelace + 300 4b6897a0af19de09282bad1b3d191ace3c61ec364aa009102f0d23b3.476c6f6265 + TxOutDatumNone


## Clean Up

In [52]:
TX_CLEANUP_TOKEN_B=$(marlowe-cli util select --asset-only "$TOKEN_B" "$PARTY_A_ADDRESS" | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/')

In [53]:
marlowe-cli transaction simple --tx-in "$TX_2#0"                                      \
                               --tx-in "$TX_CLEANUP_TOKEN_B"                          \
                               --tx-out "$PARTY_B_ADDRESS+1500000+$AMOUNT_B $TOKEN_B" \
                               --required-signer "$PARTY_A_PAYMENT_SKEY"              \
                               --change-address "$PARTY_A_ADDRESS"                    \
                               --out-file /dev/null                                   \
                               --submit 600

TxId "fd337cdbc1f5802131416ab83bc90bba6e94e5c9762d3e5fab736e443c1d8ddb"


In [54]:
TX_CLEANUP_TOKEN_A=$(marlowe-cli util select --asset-only "$TOKEN_A" "$PARTY_B_ADDRESS" | sed -e 's/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/')

In [55]:
marlowe-cli transaction simple --tx-in "$TX_3#0"                                      \
                               --tx-in "$TX_CLEANUP_TOKEN_A"                          \
                               --tx-out "$PARTY_A_ADDRESS+1500000+$AMOUNT_A $TOKEN_A" \
                               --required-signer "$PARTY_B_PAYMENT_SKEY"              \
                               --change-address "$PARTY_B_ADDRESS"                    \
                               --out-file /dev/null                                   \
                               --submit 600

TxId "4064325175127665d0131b223365d0fe72416468199a98bc347c181d9a516312"


In [56]:
marlowe-cli util mint --required-signer "$PARTY_A_PAYMENT_SKEY" \
                      --change-address  "$PARTY_A_ADDRESS"      \
                      --count "-$AMOUNT_A"                      \
                      --expires "$MINT_EXPIRES"                 \
                      --out-file /dev/null                      \
                      --submit=600                              \
                      "$TOKEN_NAME_A"

PolicyID "4b6897a0af19de09282bad1b3d191ace3c61ec364aa009102f0d23b3"


In [57]:
marlowe-cli util mint --required-signer "$PARTY_B_PAYMENT_SKEY" \
                      --change-address  "$PARTY_B_ADDRESS"      \
                      --count "-$AMOUNT_B"                      \
                      --expires "$MINT_EXPIRES"                 \
                      --out-file /dev/null                      \
                      --submit=600                              \
                      "$TOKEN_NAME_B"

PolicyID "13d32b28fbb50440a92f618434d91a6bdbf46d4da9a7838a82937492"


In [58]:
TX=$(
marlowe-cli util select --lovelace-only 1                 \
                        "$PARTY_A_ADDRESS"                \
| sed -n -e '1{s/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;p}' \
)

In [59]:
marlowe-cli transaction simple --tx-in "$TX"                             \
                               --required-signer "$PARTY_A_PAYMENT_SKEY" \
                               --change-address "$FAUCET_ADDRESS"        \
                               --out-file /dev/null                      \
                               --submit 600

TxId "17b70ac661919861927f3fd2d7ee7a984b798e18410ceb02d266d47b9500f366"


In [60]:
TX=$(
marlowe-cli util select --lovelace-only 1                 \
                        "$PARTY_B_ADDRESS"                \
| sed -n -e '1{s/^TxIn "\(.*\)" (TxIx \(.*\))$/\1#\2/;p}' \
)

In [61]:
marlowe-cli transaction simple --tx-in "$TX"                             \
                               --required-signer "$PARTY_B_PAYMENT_SKEY" \
                               --change-address "$FAUCET_ADDRESS"        \
                               --out-file /dev/null                      \
                               --submit 600

TxId "86282b22ec277ea8303eb6c41fcd35bfa459a94680b3a5629a53e6daf7a028e0"


In [62]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PARTY_A_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [63]:
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$PARTY_B_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


In [64]:
marlowe-cli util clean --required-signer "$FAUCET_SKEY_FILE" \
                       --change-address "$FAUCET_ADDRESS"    \
                       --out-file /dev/null                  \
                       --submit=600                          \
> /dev/null
cardano-cli query utxo --testnet-magic "$CARDANO_TESTNET_MAGIC" --address "$FAUCET_ADDRESS"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
56d7307ad7efe0c91f1a792b17a2e17c16898af4cbec75a6615f8fef961bc41a     0        139565033885 lovelace + TxOutDatumNone
